In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F

from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [2]:
import re

regex = re.compile(r'^[\d]+_x=([-\d.]+)_z=([-\d.]+)_a=([-\d.]+)_h=([-\d.]+)[.]csv$')

def label_from_path(path):
    label = path.split('\\')[-1] # PLATFORM DEPENDENT!
    m = regex.match(label)
    out = [float(x) for x in m.groups()]
    return torch.tensor(out, dtype=torch.float)

In [3]:
def data_from_path(path):
    data = pd.read_csv(path, sep=';')

    time = data.values[:,0]
    vx   = data.values[:,1::2]
    vz   = data.values[:,2::2]
    vabs = (vx ** 2 + vz ** 2) ** (1/2)
    
    vx   = torch.from_numpy(vx)
    vz   = torch.from_numpy(vz)
    vabs = torch.from_numpy(vabs)
    
    vx_fft   = torch.view_as_real(torch.fft.fft(vx)).permute(2, 0, 1)
    vz_fft   = torch.view_as_real(torch.fft.fft(vz)).permute(2, 0, 1)
    vabs_fft = torch.view_as_real(torch.fft.fft(vabs)).permute(2, 0, 1)

    ret = torch.cat([vx.unsqueeze(0), vz.unsqueeze(0), vabs.unsqueeze(0), vx_fft, vz_fft, vabs_fft], dim=0)

    return ret

In [4]:
class CSVDataset(Dataset):
    def __init__(self, path, device):
        self.device = device
        self.csv_list = [
            (path + "\\" + i) for i in os.listdir(path) if i.split('.')[-1] == 'csv'
        ]

    def __getitem__(self, item):
        d = data_from_path(self.csv_list[item]).to(device=device, dtype=torch.float)
        l = label_from_path(self.csv_list[item]).to(device=device, dtype=torch.float)
        return d, l

    def __len__(self):
        return len(self.csv_list)

In [5]:
class Predictor(nn.Module):
    def __init__(self):
        super(Predictor, self).__init__()

        self.feature_extractor = nn.Sequential(
            #        in  out  ker size
            nn.Conv2d(9, 32, 3),     # 9 = len([v_x, v_y, v_abs, 2*fft_vx, 2*fft_vy, 2*fft_vabs, ])
            nn.MaxPool2d(2, 2), 
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),

            nn.Conv2d(32, 512, 3, 2),
            nn.MaxPool2d(2, 2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(512),

            nn.Conv2d(512, 1024, 3, 2),
            nn.MaxPool2d(2, 2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(1024),

            nn.Conv2d(1024, 512, 2),
        )

        self.linear = nn.Linear(45, 256)
        self.conv = nn.Sequential(
            nn.Conv2d(512, 1, 1),
            nn.ReLU(inplace=True)
            )

        self.linear2 = nn.Linear(256, 4) # 4 = x, z, a, h

    def forward(self, input):
        x = self.feature_extractor(input)
        x = x.permute(0, 1, 3, 2)
        x = self.linear(x)
        x = self.conv(x)
        x = x.squeeze(1).squeeze(1)
        x = self.linear2(x)

        return x

In [6]:
predictor = Predictor().to(device)
data_loader = DataLoader(CSVDataset(".\\csv", device), batch_size=8, shuffle=True)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(predictor.parameters(), lr=0.0003)

In [7]:
from tqdm import tqdm

num_epochs = 25
all_loss = []

for epoch in range(num_epochs):
    progress = tqdm(data_loader)
    running_loss = []
    for inputs, label in progress:
        # zero the parameter gradients
        optimizer.zero_grad()
        # predict
        outputs = predictor(inputs)
        # loss
        loss = loss_fn(outputs, label)
        running_loss.append(loss.item())
        loss.backward()
        optimizer.step()

        # print statistics
        progress.set_description(f"loss: {loss.item()}")
    
    all_loss.append(running_loss)
    torch.save(predictor.state_dict(), f'.\\weights\\lastest_{epoch}_{np.mean(running_loss)}')

print('Finished Training')

loss: 673.7460327148438: 100%|███████████████████████████████████████████████████████| 750/750 [08:56<00:00,  1.40it/s]

Finished Training
